In [ ]:
#import impulsegpt_sdpa
import torch
from torch import nn
from torchinfo import summary
from transformers import AutoTokenizer
from matplotlib import pyplot as plt


In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA")
    print(torch.cuda.get_device_capability())
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS")
else:
    print("Using CPU")


In [ ]:
model = torch.load('ckpt/impgpt-64-0.pt', weights_only=False)
summary(model)

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")

In [ ]:
start_ids = torch.tensor(tokenizer.encode('Once upon a time,')).unsqueeze(dim=0).to(device=device)
max_length = 50
y = model.generate(start_ids, max_length=max_length, top_k=128, temp=1)
print(y)
txt = tokenizer.decode(y[0].tolist(), skip_special_tokens=True)
print(y.shape)
print(txt)

In [ ]:
y = model(start_ids)
prob = nn.functional.softmax(y, dim=-1).cpu().detach().squeeze()
token_max = torch.argmax(prob)
print(token_max)
plt.plot(prob)
tokenizer.decode([token_max.tolist()])